In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OrdinalEncoder

from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import log_loss,roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.calibration import CalibratedClassifierCV

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from prettytable import PrettyTable

import pickle
from prettytable import PrettyTable
from tqdm import tqdm
import sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('in-vehicle-coupon-recommendation.csv')
data.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


In [3]:
print("Number of data points:", data.shape[0])
print("Number of features:", data.shape[1])
print('-'*100)
print("The attributes of data :", data.columns.values)

Number of data points: 12684
Number of features: 26
----------------------------------------------------------------------------------------------------
The attributes of data : ['destination' 'passanger' 'weather' 'temperature' 'time' 'coupon'
 'expiration' 'gender' 'age' 'maritalStatus' 'has_children' 'education'
 'occupation' 'income' 'car' 'Bar' 'CoffeeHouse' 'CarryAway'
 'RestaurantLessThan20' 'Restaurant20To50' 'toCoupon_GEQ5min'
 'toCoupon_GEQ15min' 'toCoupon_GEQ25min' 'direction_same' 'direction_opp'
 'Y']


In [4]:
Y_value_counts = data.groupby('Y').Y.count()
print('The number of users that are accepted the coupon is ',Y_value_counts[1],',',round(Y_value_counts[1]/data.shape[0]*100,3),'%')
print('The number of users that are rejected the coupon is ',Y_value_counts[0],',',round(Y_value_counts[0]/data.shape[0]*100,3),'%')

The number of users that are accepted the coupon is  7210 , 56.843 %
The number of users that are rejected the coupon is  5474 , 43.157 %


## Train Test Split

In [5]:
X = data.drop(['Y'], axis=1)
y = data['Y'].values

In [6]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

In [7]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(10147, 25) (10147,)
(2537, 25) (2537,)


In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10147 entries, 1753 to 6971
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           10147 non-null  object
 1   passanger             10147 non-null  object
 2   weather               10147 non-null  object
 3   temperature           10147 non-null  int64 
 4   time                  10147 non-null  object
 5   coupon                10147 non-null  object
 6   expiration            10147 non-null  object
 7   gender                10147 non-null  object
 8   age                   10147 non-null  object
 9   maritalStatus         10147 non-null  object
 10  has_children          10147 non-null  int64 
 11  education             10147 non-null  object
 12  occupation            10147 non-null  object
 13  income                10147 non-null  object
 14  car                   90 non-null     object
 15  Bar                   10060 non-nu

## Data Cleaning

In [9]:
# Remove duplicates
# duplicate = data[data.duplicated(keep = 'last')]
# # duplicate.shape #(74, 26)
# data = data.drop_duplicates()
# print(data.shape)

#### Missing Values

In [10]:
print('Is there any missing value present or not?',data.isnull().values.any())
missing_percentage = data.isnull().sum()*100/len(data)
missing_value_df = pd.DataFrame({'missing_count': data.isnull().sum(),'missing_percentage': missing_percentage})
missing_value_df[missing_value_df.missing_count != 0]

Is there any missing value present or not? True


,missing_count,missing_percentage
car,12576,99.148534
Bar,107,0.843582
CoffeeHouse,217,1.710817
CarryAway,151,1.190476
RestaurantLessThan20,130,1.024913
Restaurant20To50,189,1.490066


In [11]:
X_train = X_train.drop(['car'], axis=1)
X_test = X_test.drop(['car'], axis=1)

#### Correlation of Features

In [12]:
X_train.corr()

,temperature,has_children,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp
temperature,1.000000,-0.023556,NaN,-0.146644,-0.205937,0.092944,-0.092944
has_children,-0.023556,1.000000,NaN,0.081037,-0.013570,-0.036979,0.036979
toCoupon_GEQ5min,NaN,NaN,NaN,NaN,NaN,NaN,NaN
toCoupon_GEQ15min,-0.146644,0.081037,NaN,1.000000,0.322756,-0.300537,0.300537
toCoupon_GEQ25min,-0.205937,-0.013570,NaN,0.322756,1.000000,-0.190973,0.190973
direction_same,0.092944,-0.036979,NaN,-0.300537,-0.190973,1.000000,-1.000000
direction_opp,-0.092944,0.036979,NaN,0.300537,0.190973,-1.000000,1.000000


  1.Feature ‘direction_same’ is perfectly correlated with ‘direction_opp’, both     have the same variance.
  
  2.‘toCoupon_GEQ5min’ feature has no correlation with any feature because it        has the same value ‘1’ for all data points, which means all the                restaurants/bars are at least more than five minutes away from the driver.
  
  so, drop both 'direction_opp' and 'toCoupon_GEQ5min' features.

In [13]:
X_train = X_train.drop(['direction_opp','toCoupon_GEQ5min'], axis=1)
X_test = X_test.drop(['direction_opp','toCoupon_GEQ5min'], axis=1)
X_train.shape, X_test.shape

((10147, 22), (2537, 22))

In [14]:
X_train.describe()

,temperature,has_children,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same
count,10147.000000,10147.000000,10147.000000,10147.000000,10147.000000
mean,63.354686,0.415295,0.561447,0.117670,0.214743
std,19.160522,0.492797,0.496234,0.322233,0.410664
min,30.000000,0.000000,0.000000,0.000000,0.000000
25%,55.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,0.000000,1.000000,0.000000,0.000000
75%,80.000000,1.000000,1.000000,0.000000,0.000000
max,80.000000,1.000000,1.000000,1.000000,1.000000


## Mode Imputation

In [15]:
print('Is there any missing value present or not?',X_train.isnull().values.any())

Is there any missing value present or not? True


In [16]:
# mode imputation for missing values in train data
X_train['Bar'] = X_train['Bar'].fillna(X_train['Bar'].value_counts().index[0])
X_train['CoffeeHouse'] = X_train['CoffeeHouse'].fillna(X_train['CoffeeHouse'].value_counts().index[0])
X_train['CarryAway'] = X_train['CarryAway'].fillna(X_train['CarryAway'].value_counts().index[0])
X_train['RestaurantLessThan20'] = X_train['RestaurantLessThan20'].fillna(X_train['RestaurantLessThan20'].value_counts().index[0])
X_train['Restaurant20To50'] = X_train['Restaurant20To50'].fillna(X_train['Restaurant20To50'].value_counts().index[0])

In [17]:
# mode imputation for missing values in test data
X_test['Bar'] = X_test['Bar'].fillna(X_train['Bar'].value_counts().index[0])
X_test['CoffeeHouse'] = X_test['CoffeeHouse'].fillna(X_train['CoffeeHouse'].value_counts().index[0])
X_test['CarryAway'] = X_test['CarryAway'].fillna(X_train['CarryAway'].value_counts().index[0])
X_test['RestaurantLessThan20'] = X_test['RestaurantLessThan20'].fillna(X_train['RestaurantLessThan20'].value_counts().index[0])
X_test['Restaurant20To50'] = X_test['Restaurant20To50'].fillna(X_train['Restaurant20To50'].value_counts().index[0])

In [18]:
print('Is there any missing value present in X_train?',X_train.isnull().values.any())

Is there any missing value present in X_train? False


In [19]:
print('Is there any missing value present in X_test?',X_test.isnull().values.any())

Is there any missing value present in X_test? False


## Feature Engineering

In [20]:
# FE -- to_Coupon is combination of two features, toCoupon_GEQ15min and toCoupon_GEQ25min
to_Coupon = []
for i in range(X_train.shape[0]):
    if (list(X_train['toCoupon_GEQ15min'])[i] == 0):
        to_Coupon.append(0)
    elif (list(X_train['toCoupon_GEQ15min'])[i] == 1)and(list(X_train['toCoupon_GEQ25min'])[i] == 0):
        to_Coupon.append(1)
    else:
        to_Coupon.append(2)
        
X_train['to_Coupon'] = to_Coupon
print('Unique values:',X_train['to_Coupon'].unique())
print('-'*50)
X_train['to_Coupon'].describe()

Unique values: [0 1 2]
--------------------------------------------------


count    10147.000000
mean         0.679117
std          0.673277
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          2.000000
Name: to_Coupon, dtype: float64

In [21]:
# FE -- to_Coupon is combination of two features, toCoupon_GEQ15min and toCoupon_GEQ25min
to_Coupon = []
for i in range(X_test.shape[0]):
    if (list(X_test['toCoupon_GEQ15min'])[i] == 0):
        to_Coupon.append(0)
    elif (list(X_test['toCoupon_GEQ15min'])[i] == 1)and(list(X_test['toCoupon_GEQ25min'])[i] == 0):
        to_Coupon.append(1)
    else:
        to_Coupon.append(2)
        
X_test['to_Coupon'] = to_Coupon
print('Unique values:',X_test['to_Coupon'].unique())
print('-'*50)
X_test['to_Coupon'].describe()

Unique values: [0 1 2]
--------------------------------------------------


count    2537.000000
mean        0.686638
std         0.682093
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: to_Coupon, dtype: float64

In [22]:
# FE -- coupon_freq is combination of five features, RestaurantLessThan20, CoffeeHouse, CarryAway, Bar, Restaurant20To50
coupon_freq = []
for i in range(X_train.shape[0]):
    if (list(X_train['coupon'])[i] == 'Restaurant(<20)'):
        coupon_freq.append(list(X_train['RestaurantLessThan20'])[i])
    elif (list(X_train['coupon'])[i] == 'Coffee House'):
        coupon_freq.append(list(X_train['CoffeeHouse'])[i])
    elif (list(X_train['coupon'])[i] == 'Carry out & Take away'):
        coupon_freq.append(list(X_train['CarryAway'])[i])
    elif (list(X_train['coupon'])[i] == 'Bar'):
        coupon_freq.append(list(X_train['Bar'])[i])
    elif (list(X_train['coupon'])[i] == 'Restaurant(20-50)'):
        coupon_freq.append(list(X_train['Restaurant20To50'])[i])
        
X_train['coupon_freq'] = coupon_freq
print('Unique values:',X_train['coupon_freq'].unique())
print('-'*50)
X_train['coupon_freq'].describe()

Unique values: ['1~3' '4~8' 'gt8' 'less1' 'never']
--------------------------------------------------


count     10147
unique        5
top         1~3
freq       3118
Name: coupon_freq, dtype: object

In [23]:
# FE -- coupon_freq is combination of five features, RestaurantLessThan20, CoffeeHouse, CarryAway, Bar, Restaurant20To50
coupon_freq = []
for i in range(X_test.shape[0]):
    if (list(X_test['coupon'])[i] == 'Restaurant(<20)'):
        coupon_freq.append(list(X_test['RestaurantLessThan20'])[i])
    elif (list(X_test['coupon'])[i] == 'Coffee House'):
        coupon_freq.append(list(X_test['CoffeeHouse'])[i])
    elif (list(X_test['coupon'])[i] == 'Carry out & Take away'):
        coupon_freq.append(list(X_test['CarryAway'])[i])
    elif (list(X_test['coupon'])[i] == 'Bar'):
        coupon_freq.append(list(X_test['Bar'])[i])
    elif (list(X_test['coupon'])[i] == 'Restaurant(20-50)'):
        coupon_freq.append(list(X_test['Restaurant20To50'])[i])
        
X_test['coupon_freq'] = coupon_freq
print('Unique values:',X_test['coupon_freq'].unique()) 
print('-'*50)
X_test['coupon_freq'].describe()

Unique values: ['4~8' 'less1' 'never' '1~3' 'gt8']
--------------------------------------------------


count     2537
unique       5
top        1~3
freq       774
Name: coupon_freq, dtype: object

In [24]:
X_train['occupation'].describe()

count          10147
unique            25
top       Unemployed
freq            1525
Name: occupation, dtype: object

In [25]:
# occupation feature has 25 no of distinct values, which creates very sparsity in data after Encoding
# FE -- occupation_class where categorize all occupation in its suitable class.
occupation_dict = {'Healthcare Support':'High_Acceptance','Construction & Extraction':'High_Acceptance','Healthcare Practitioners & Technical':'High_Acceptance',
                   'Protective Service':'High_Acceptance','Architecture & Engineering':'High_Acceptance','Production Occupations':'Medium_High_Acceptance',
                    'Student':'Medium_High_Acceptance','Office & Administrative Support':'Medium_High_Acceptance','Transportation & Material Moving':'Medium_High_Acceptance',
                    'Building & Grounds Cleaning & Maintenance':'Medium_High_Acceptance','Management':'Medium_Acceptance','Food Preparation & Serving Related':'Medium_Acceptance',
                   'Life Physical Social Science':'Medium_Acceptance','Business & Financial':'Medium_Acceptance','Computer & Mathematical':'Medium_Acceptance',
                    'Sales & Related':'Medium_Low_Acceptance','Personal Care & Service':'Medium_Low_Acceptance','Unemployed':'Medium_Low_Acceptance',
                   'Farming Fishing & Forestry':'Medium_Low_Acceptance','Installation Maintenance & Repair':'Medium_Low_Acceptance','Education&Training&Library':'Low_Acceptance',
                    'Arts Design Entertainment Sports & Media':'Low_Acceptance','Community & Social Services':'Low_Acceptance','Legal':'Low_Acceptance','Retired':'Low_Acceptance'}
# occupation_dict
X_train['occupation_class'] = X_train['occupation'].map(occupation_dict)
print('Unique values:',X_train['occupation_class'].unique())
print('-'*50)
X_train['occupation_class'].describe()
# X_train['occupation_class'].value_counts()

Unique values: ['Low_Acceptance' 'Medium_High_Acceptance' 'Medium_Low_Acceptance'
 'Medium_Acceptance' 'High_Acceptance']
--------------------------------------------------


count                     10147
unique                        5
top       Medium_Low_Acceptance
freq                       2672
Name: occupation_class, dtype: object

In [26]:
X_test['occupation'].describe()

count           2537
unique            25
top       Unemployed
freq             345
Name: occupation, dtype: object

In [27]:
# occupation feature has 25 no of distinct values, which creates very sparsity in data after Encoding
# FE -- occupation_class where categorize all occupation in its suitable class.
occupation_dict = {'Healthcare Support':'High_Acceptance','Construction & Extraction':'High_Acceptance','Healthcare Practitioners & Technical':'High_Acceptance',
                   'Protective Service':'High_Acceptance','Architecture & Engineering':'High_Acceptance','Production Occupations':'Medium_High_Acceptance',
                    'Student':'Medium_High_Acceptance','Office & Administrative Support':'Medium_High_Acceptance','Transportation & Material Moving':'Medium_High_Acceptance',
                    'Building & Grounds Cleaning & Maintenance':'Medium_High_Acceptance','Management':'Medium_Acceptance','Food Preparation & Serving Related':'Medium_Acceptance',
                   'Life Physical Social Science':'Medium_Acceptance','Business & Financial':'Medium_Acceptance','Computer & Mathematical':'Medium_Acceptance',
                    'Sales & Related':'Medium_Low_Acceptance','Personal Care & Service':'Medium_Low_Acceptance','Unemployed':'Medium_Low_Acceptance',
                   'Farming Fishing & Forestry':'Medium_Low_Acceptance','Installation Maintenance & Repair':'Medium_Low_Acceptance','Education&Training&Library':'Low_Acceptance',
                    'Arts Design Entertainment Sports & Media':'Low_Acceptance','Community & Social Services':'Low_Acceptance','Legal':'Low_Acceptance','Retired':'Low_Acceptance'}
# occupation_dict
X_test['occupation_class'] = X_test['occupation'].map(occupation_dict)
print('Unique values:',X_test['occupation_class'].unique())
print('-'*50)
X_test['occupation_class'].describe()
# X_test['occupation_class'].value_counts()

Unique values: ['Medium_Acceptance' 'High_Acceptance' 'Low_Acceptance'
 'Medium_High_Acceptance' 'Medium_Low_Acceptance']
--------------------------------------------------


count                  2537
unique                    5
top       Medium_Acceptance
freq                    679
Name: occupation_class, dtype: object

In [28]:
# After Feature Engineering, removing unwanted features
# X_train = X_train.drop(['toCoupon_GEQ15min','toCoupon_GEQ25min','Bar','CoffeeHouse','CarryAway','RestaurantLessThan20','Restaurant20To50','occupation'], axis=1)
# X_test = X_test.drop(['toCoupon_GEQ15min','toCoupon_GEQ25min','Bar','CoffeeHouse','CarryAway','RestaurantLessThan20','Restaurant20To50','occupation'], axis=1)
X_train = X_train.drop(['occupation'], axis=1)
X_test = X_test.drop(['occupation'], axis=1)
print('X_train:',X_train.shape,'\nX_test:',X_test.shape)
print('-'*50)
print(X_train.columns.values)

X_train: (10147, 24) 
X_test: (2537, 24)
--------------------------------------------------
['destination' 'passanger' 'weather' 'temperature' 'time' 'coupon'
 'expiration' 'gender' 'age' 'maritalStatus' 'has_children' 'education'
 'income' 'Bar' 'CoffeeHouse' 'CarryAway' 'RestaurantLessThan20'
 'Restaurant20To50' 'toCoupon_GEQ15min' 'toCoupon_GEQ25min'
 'direction_same' 'to_Coupon' 'coupon_freq' 'occupation_class']


## Encoding

### Ordinal Encoding

In [29]:
order = [['Work','Home','No Urgent Place'],['Kid(s)','Alone','Partner','Friend(s)'],['Rainy','Snowy','Sunny'],[30,55,80],['7AM','10AM','2PM','6PM','10PM'],
         ['Bar','Restaurant(20-50)','Coffee House','Restaurant(<20)','Carry out & Take away'],['2h','1d'],['Female','Male'],['below21','21','26','31','36','41','46','50plus'],
         ['Widowed','Divorced','Married partner','Unmarried partner','Single'],[0,1],
         ['Some High School','High School Graduate','Some college - no degree','Associates degree','Bachelors degree','Graduate degree (Masters or Doctorate)'],
         ['Less than $12500','$12500 - $24999','$25000 - $37499','$37500 - $49999','$50000 - $62499','$62500 - $74999','$75000 - $87499','$87500 - $99999','$100000 or More'],
         ['never','less1','1~3','4~8','gt8'],['never','less1','1~3','4~8','gt8'],['never','less1','1~3','4~8','gt8'],['never','less1','1~3','4~8','gt8'],['never','less1','1~3','4~8','gt8'],
         [0,1],[0,1],[0,1],[0,1,2],['never','less1','1~3','4~8','gt8'],['Low_Acceptance','Medium_Low_Acceptance','Medium_Acceptance','Medium_High_Acceptance','High_Acceptance']]

In [30]:
Ordinal_enc = OrdinalEncoder(categories=order)
X_train_Ordinal_encoding = Ordinal_enc.fit_transform(X_train)
X_train_Ordinal_encoding = pd.DataFrame(X_train_Ordinal_encoding,columns=X_train.columns.values)
print('X_train_Ordinal_encoding:',X_train_Ordinal_encoding.shape)

X_train_Ordinal_encoding: (10147, 24)


In [31]:
Ordinal_enc = OrdinalEncoder(categories=order)
X_test_Ordinal_encoding = Ordinal_enc.fit_transform(X_test)
X_test_Ordinal_encoding = pd.DataFrame(X_test_Ordinal_encoding,columns=X_test.columns.values)
print('X_test_Ordinal_encoding:',X_test_Ordinal_encoding.shape)

X_test_Ordinal_encoding: (2537, 24)


### Frequency Encoding

In [32]:
def frequency_enc(column_name,X):
  """It returns Frequency encoded feature"""
  return X[column_name].map(X.groupby(column_name).size()/len(X))

In [33]:
X_train_frequency_encoding = pd.DataFrame()
for i in range(X_train.shape[1]):
  X_train_frequency_encoding[X_train.columns.values[i]+'_freq_enc'] = frequency_enc(X_train.columns.values[i],X_train)

print('X_train_frequency_encoding:',X_train_frequency_encoding.shape)

X_train_frequency_encoding: (10147, 24)


In [34]:
X_test_frequency_encoding = pd.DataFrame()
for i in range(X_test.shape[1]):
  X_test_frequency_encoding[X_test.columns.values[i]+'_freq_enc'] = frequency_enc(X_test.columns.values[i],X_test)

print('X_test_frequency_encoding:',X_test_frequency_encoding.shape)

X_test_frequency_encoding: (2537, 24)


### Target Encoding

In [35]:
def target_enc(column_name,X):
  """It returns Target encoded feature for train data"""
  X['Y_train'] = y_train
  return X[column_name].map(X.groupby(column_name)['Y_train'].mean())

X_train_target_encoding = pd.DataFrame()
for i in range(X_train.shape[1]):
  X_train_target_encoding[X_train.columns.values[i]+'_target_enc'] = target_enc(X_train.columns.values[i],X_train)

print('X_train_target_encoding:',X_train_target_encoding.shape)

X_train_target_encoding: (10147, 24)


In [36]:
def target_enc(column_name,X):
  """It returns Target encoded feature for test data"""
  X['Y_test'] = y_test
  return X[column_name].map(X.groupby(column_name)['Y_test'].mean())

X_test_target_encoding = pd.DataFrame()
for i in range(X_test.shape[1]):
  X_test_target_encoding[X_test.columns.values[i]+'_target_enc'] = target_enc(X_test.columns.values[i],X_test)

print('X_test_target_encoding:',X_test_target_encoding.shape)

X_test_target_encoding: (2537, 24)


### Response Encoding

In [37]:
# response encoding function
def response_coding(feature,X,Y):
    """It returns Response encoded feature"""
    X[feature] = X[feature].str.replace('~','_')
    X[feature] = X[feature].str.replace('[^a-zA-Z0-9_ ]',' ')
    X[feature] = X[feature].str.replace(' +',' ')
    X[feature] = X[feature].str.strip()
    X[feature] = X[feature].str.replace(' ','_')
    X[feature] = X[feature].str.lower()
    response_code_0 = [];response_code_1 = []
    unique_cat_features = X[feature].unique()
    unique_cat_features = np.sort(unique_cat_features)
    for i in range(len(unique_cat_features)):
        total_count = X[feature][(X[feature] == unique_cat_features[i])].count()
        p0 = (X[feature][((X[feature] == unique_cat_features[i]) & (Y==0))].count())/total_count
        p1 = (X[feature][((X[feature] == unique_cat_features[i]) & (Y==1))].count())/total_count
        response_code_0.append(p0);response_code_1.append(p1)
    dict_response_code_0 = dict(zip(unique_cat_features, response_code_0))
    dict_response_code_1 = dict(zip(unique_cat_features, response_code_1))
    X_response_0 = X[feature].map(dict_response_code_0)
    X_response_1 = X[feature].map(dict_response_code_1)
    X_response_0 = X_response_0.values.reshape(-1,1)
    X_response_1 = X_response_1.values.reshape(-1,1)
    return X_response_0,X_response_1

In [38]:
X_train_destination_0,X_train_destination_1 = response_coding('destination',X_train,y_train)
X_train_passanger_0,X_train_passanger_1 = response_coding('passanger',X_train,y_train)
X_train_weather_0,X_train_weather_1 = response_coding('weather',X_train,y_train)
X_train_time_0,X_train_time_1 = response_coding('time',X_train,y_train)
X_train_coupon_0,X_train_coupon_1 = response_coding('coupon',X_train,y_train)
X_train_expiration_0,X_train_expiration_1 = response_coding('expiration',X_train,y_train)
X_train_gender_0,X_train_gender_1 = response_coding('gender',X_train,y_train)
X_train_age_0,X_train_age_1 = response_coding('age',X_train,y_train)
X_train_maritalStatus_0,X_train_maritalStatus_1 = response_coding('maritalStatus',X_train,y_train)
X_train_education_0,X_train_education_1 = response_coding('education',X_train,y_train)
X_train_income_0,X_train_income_1 = response_coding('income',X_train,y_train)
X_train_Bar_0,X_train_Bar_1 = response_coding('Bar',X_train,y_train)
X_train_CoffeeHouse_0,X_train_CoffeeHouse_1 = response_coding('CoffeeHouse',X_train,y_train)
X_train_CarryAway_0,X_train_CarryAway_1 = response_coding('CarryAway',X_train,y_train)
X_train_RestaurantLessThan20_0,X_train_RestaurantLessThan20_1 = response_coding('RestaurantLessThan20',X_train,y_train)
X_train_Restaurant20To50_0,X_train_Restaurant20To50_1 = response_coding('Restaurant20To50',X_train,y_train)
X_train_coupon_freq_0,X_train_coupon_freq_1 = response_coding('coupon_freq',X_train,y_train)
X_train_occupation_class_0,X_train_occupation_class_1 = response_coding('occupation_class',X_train,y_train)

X_test_destination_0,X_test_destination_1 = response_coding('destination',X_test,y_test)
X_test_passanger_0,X_test_passanger_1 = response_coding('passanger',X_test,y_test)
X_test_weather_0,X_test_weather_1 = response_coding('weather',X_test,y_test)
X_test_time_0,X_test_time_1 = response_coding('time',X_test,y_test)
X_test_coupon_0,X_test_coupon_1 = response_coding('coupon',X_test,y_test)
X_test_expiration_0,X_test_expiration_1 = response_coding('expiration',X_test,y_test)
X_test_gender_0,X_test_gender_1 = response_coding('gender',X_test,y_test)
X_test_age_0,X_test_age_1 = response_coding('age',X_test,y_test)
X_test_maritalStatus_0,X_test_maritalStatus_1 = response_coding('maritalStatus',X_test,y_test)
X_test_education_0,X_test_education_1 = response_coding('education',X_test,y_test)
X_test_income_0,X_test_income_1 = response_coding('income',X_test,y_test)
X_test_Bar_0,X_test_Bar_1 = response_coding('Bar',X_test,y_test)
X_test_CoffeeHouse_0,X_test_CoffeeHouse_1 = response_coding('CoffeeHouse',X_test,y_test)
X_test_CarryAway_0,X_test_CarryAway_1 = response_coding('CarryAway',X_test,y_test)
X_test_RestaurantLessThan20_0,X_test_RestaurantLessThan20_1 = response_coding('RestaurantLessThan20',X_test,y_test)
X_test_Restaurant20To50_0,X_test_Restaurant20To50_1 = response_coding('Restaurant20To50',X_test,y_test)
X_test_coupon_freq_0,X_test_coupon_freq_1 = response_coding('coupon_freq',X_test,y_test)
X_test_occupation_class_0,X_test_occupation_class_1 = response_coding('occupation_class',X_test,y_test)

In [39]:
# Normalization of numerical features
def norm(column_name,X):
    """It returns Normalized feature"""
    normalizer = Normalizer()
    normalizer.fit(X[column_name].values.reshape(1,-1))
    X_norm = normalizer.transform(X[column_name].values.reshape(1,-1))
    return X_norm.reshape(-1,1)

In [40]:
X_train_temperature_norm = norm('temperature',X_train)
X_train_has_children_norm = norm('has_children',X_train)
X_train_toCoupon_GEQ15min_norm = norm('toCoupon_GEQ15min',X_train)
X_train_toCoupon_GEQ25min_norm = norm('toCoupon_GEQ25min',X_train)
X_train_direction_same_norm = norm('direction_same',X_train)
X_train_to_Coupon_norm = norm('to_Coupon',X_train)

X_test_temperature_norm = norm('temperature',X_test)
X_test_has_children_norm = norm('has_children',X_test)
X_test_toCoupon_GEQ15min_norm = norm('toCoupon_GEQ15min',X_test)
X_test_toCoupon_GEQ25min_norm = norm('toCoupon_GEQ25min',X_test)
X_test_direction_same_norm = norm('direction_same',X_test)
X_test_to_Coupon_norm = norm('to_Coupon',X_test)

In [41]:
X_train_response_encoding = np.hstack((X_train_destination_0,X_train_destination_1,X_train_passanger_0,X_train_passanger_1,X_train_weather_0,X_train_weather_1,X_train_time_0,X_train_time_1,X_train_coupon_0,X_train_coupon_1,X_train_expiration_0,X_train_expiration_1,X_train_gender_0,X_train_gender_1,X_train_age_0,X_train_age_1,X_train_maritalStatus_0,X_train_maritalStatus_1,X_train_education_0,X_train_education_1,X_train_income_0,X_train_income_1,X_train_coupon_freq_0,X_train_coupon_freq_1,X_train_occupation_class_0,X_train_occupation_class_1,X_train_Bar_0,X_train_Bar_1,X_train_CoffeeHouse_0,X_train_CoffeeHouse_1,X_train_CarryAway_0,X_train_CarryAway_1,X_train_RestaurantLessThan20_0,X_train_RestaurantLessThan20_1,X_train_Restaurant20To50_0,X_train_Restaurant20To50_1,X_train_temperature_norm,X_train_has_children_norm,X_train_toCoupon_GEQ15min_norm,X_train_toCoupon_GEQ25min_norm,X_train_direction_same_norm,X_train_to_Coupon_norm))
X_test_response_encoding = np.hstack((X_test_destination_0,X_test_destination_1,X_test_passanger_0,X_test_passanger_1,X_test_weather_0,X_test_weather_1,X_test_time_0,X_test_time_1,X_test_coupon_0,X_test_coupon_1,X_test_expiration_0,X_test_expiration_1,X_test_gender_0,X_test_gender_1,X_test_age_0,X_test_age_1,X_test_maritalStatus_0,X_test_maritalStatus_1,X_test_education_0,X_test_education_1,X_test_income_0,X_test_income_1,X_test_coupon_freq_0,X_test_coupon_freq_1,X_test_occupation_class_0,X_test_occupation_class_1,X_test_Bar_0,X_test_Bar_1,X_test_CoffeeHouse_0,X_test_CoffeeHouse_1,X_test_CarryAway_0,X_test_CarryAway_1,X_test_RestaurantLessThan20_0,X_test_RestaurantLessThan20_1,X_test_Restaurant20To50_0,X_test_Restaurant20To50_1 ,X_test_temperature_norm,X_test_has_children_norm,X_test_toCoupon_GEQ15min_norm,X_test_toCoupon_GEQ25min_norm,X_test_direction_same_norm,X_test_to_Coupon_norm))
print('X_train_response_encoding:',X_train_response_encoding.shape,'\nX_test_response_encoding:',X_test_response_encoding.shape)

X_train_response_encoding: (10147, 42) 
X_test_response_encoding: (2537, 42)


### One Hot Encoding

In [42]:
# one hot encoding function
def ohe(column_name,X):
    """It returns One hot encoded feature in X data"""  
    X[column_name] = X[column_name].str.replace('~','_')
    X[column_name] = X[column_name].str.replace('[^a-zA-Z0-9_ ]',' ')
    X[column_name] = X[column_name].str.replace(' +',' ')
    X[column_name] = X[column_name].str.strip()
    X[column_name] = X[column_name].str.replace(' ','_')
    X[column_name] = X[column_name].str.lower()
    vectorizer = CountVectorizer(binary=True)
    return vectorizer.fit_transform(X[column_name].values)

In [43]:
X_train_destination_ohe = ohe('destination',X_train)
X_train_passanger_ohe = ohe('passanger',X_train)
X_train_weather_ohe = ohe('weather',X_train)
X_train_time_ohe = ohe('time',X_train)
X_train_coupon_ohe = ohe('coupon',X_train)
X_train_expiration_ohe = ohe('expiration',X_train)
X_train_gender_ohe = ohe('gender',X_train)
X_train_age_ohe = ohe('age',X_train)
X_train_maritalStatus_ohe = ohe('maritalStatus',X_train)
X_train_education_ohe = ohe('education',X_train)
X_train_income_ohe = ohe('income',X_train)
X_train_Bar_ohe = ohe('Bar',X_train)
X_train_CoffeeHouse_ohe = ohe('CoffeeHouse',X_train)
X_train_CarryAway_ohe = ohe('CarryAway',X_train)
X_train_RestaurantLessThan20_ohe = ohe('RestaurantLessThan20',X_train)
X_train_Restaurant20To50_ohe = ohe('Restaurant20To50',X_train)
X_train_coupon_freq_ohe = ohe('coupon_freq',X_train)
X_train_occupation_class_ohe = ohe('occupation_class',X_train)

X_test_destination_ohe = ohe('destination',X_test)
X_test_passanger_ohe = ohe('passanger',X_test)
X_test_weather_ohe = ohe('weather',X_test)
X_test_time_ohe = ohe('time',X_test)
X_test_coupon_ohe = ohe('coupon',X_test)
X_test_expiration_ohe = ohe('expiration',X_test)
X_test_gender_ohe = ohe('gender',X_test)
X_test_age_ohe = ohe('age',X_test)
X_test_maritalStatus_ohe = ohe('maritalStatus',X_test)
X_test_education_ohe = ohe('education',X_test)
X_test_income_ohe = ohe('income',X_test)
X_test_Bar_ohe = ohe('Bar',X_test)
X_test_CoffeeHouse_ohe = ohe('CoffeeHouse',X_test)
X_test_CarryAway_ohe = ohe('CarryAway',X_test)
X_test_RestaurantLessThan20_ohe = ohe('RestaurantLessThan20',X_test)
X_test_Restaurant20To50_ohe = ohe('Restaurant20To50',X_test)
X_test_coupon_freq_ohe = ohe('coupon_freq',X_test)
X_test_occupation_class_ohe = ohe('occupation_class',X_test)

In [44]:
# Normalization of numerical features
def norm(column_name,X):
    """It returns Normalized feature"""
    normalizer = Normalizer()
    normalizer.fit(X[column_name].values.reshape(1,-1))
    X_norm = normalizer.transform(X[column_name].values.reshape(1,-1))
    return X_norm.reshape(-1,1)

In [45]:
X_train_temperature_norm = norm('temperature',X_train)
X_train_has_children_norm = norm('has_children',X_train)
X_train_toCoupon_GEQ15min_norm = norm('toCoupon_GEQ15min',X_train)
X_train_toCoupon_GEQ25min_norm = norm('toCoupon_GEQ25min',X_train)
X_train_direction_same_norm = norm('direction_same',X_train)
X_train_to_Coupon_norm = norm('to_Coupon',X_train)

X_test_temperature_norm = norm('temperature',X_test)
X_test_has_children_norm = norm('has_children',X_test)
X_test_toCoupon_GEQ15min_norm = norm('toCoupon_GEQ15min',X_test)
X_test_toCoupon_GEQ25min_norm = norm('toCoupon_GEQ25min',X_test)
X_test_direction_same_norm = norm('direction_same',X_test)
X_test_to_Coupon_norm = norm('to_Coupon',X_test)

In [46]:
from scipy.sparse import hstack
X_train_ohe = hstack((X_train_destination_ohe, X_train_passanger_ohe, X_train_weather_ohe, X_train_time_ohe, X_train_coupon_ohe, X_train_expiration_ohe, X_train_gender_ohe, X_train_age_ohe, X_train_maritalStatus_ohe, X_train_education_ohe, X_train_income_ohe, X_train_coupon_freq_ohe, X_train_occupation_class_ohe,X_train_Bar_ohe,X_train_CoffeeHouse_ohe,X_train_CarryAway_ohe,X_train_RestaurantLessThan20_ohe,X_train_Restaurant20To50_ohe,X_train_temperature_norm, X_train_has_children_norm,X_train_toCoupon_GEQ15min_norm,X_train_toCoupon_GEQ25min_norm,X_train_direction_same_norm,X_train_to_Coupon_norm)).tocsr()
X_test_ohe = hstack((X_test_destination_ohe, X_test_passanger_ohe, X_test_weather_ohe, X_test_time_ohe, X_test_coupon_ohe, X_test_expiration_ohe, X_test_gender_ohe, X_test_age_ohe, X_test_maritalStatus_ohe, X_test_education_ohe, X_test_income_ohe, X_test_coupon_freq_ohe, X_test_occupation_class_ohe,X_test_Bar_ohe,X_test_CoffeeHouse_ohe,X_test_CarryAway_ohe,X_test_RestaurantLessThan20_ohe,X_test_Restaurant20To50_ohe,X_test_temperature_norm, X_test_has_children_norm,X_test_toCoupon_GEQ15min_norm,X_test_toCoupon_GEQ25min_norm,X_test_direction_same_norm,X_test_to_Coupon_norm)).tocsr()
print('X_train_ohe:',X_train_ohe.shape,'\nX_test_ohe:',X_test_ohe.shape)

X_train_ohe: (10147, 93) 
X_test_ohe: (2537, 93)


## Modeling

### Decision Tree

In [47]:
def Decision_Tree_Classifier(x_train,y_train,x_test,y_test):
  """This function returns best hyperparameter, train and test log_loss & roc_auc_score of Decision Trees Model"""

  clf = DecisionTreeClassifier(class_weight='balanced')
  parameters = {'max_depth':[1, 5, 10, 50], 'min_samples_split':[5, 10, 100, 500]}
  model = RandomizedSearchCV(clf, parameters, cv=5, scoring='roc_auc') #scoring='roc_auc' or 'neg_log_loss'
  model.fit(x_train, y_train)
  best_depth = model.best_params_['max_depth']
  best_samples_split = model.best_params_['min_samples_split']

  clf = DecisionTreeClassifier(class_weight='balanced', max_depth=best_depth, min_samples_split=best_samples_split, random_state=0)
  clf.fit(x_train, y_train)

  Train_loss = log_loss(y_train,clf.predict_proba(x_train))
  Train_AUC = roc_auc_score(y_train,clf.predict_proba(x_train)[:,1])
  Test_loss = log_loss(y_test,clf.predict_proba(x_test))
  Test_AUC = roc_auc_score(y_test,clf.predict_proba(x_test)[:,1])

  return best_depth,best_samples_split,Train_loss,Train_AUC,Test_loss,Test_AUC

In [48]:
# Ordinal Encoding
best_depth_OrEnc,best_samples_split_OrEnc,Train_loss_OrEnc,Train_AUC_OrEnc,Test_loss_OrEnc,Test_AUC_OrEnc = Decision_Tree_Classifier(X_train_Ordinal_encoding,y_train,X_test_Ordinal_encoding,y_test)
# Frequency Encoding
best_depth_FreEnc,best_samples_split_FreEnc,Train_loss_FreEnc,Train_AUC_FreEnc,Test_loss_FreEnc,Test_AUC_FreEnc = Decision_Tree_Classifier(X_train_frequency_encoding,y_train,X_test_frequency_encoding,y_test)
# Target Encoding
best_depth_TarEnc,best_samples_split_TarEnc,Train_loss_TarEnc,Train_AUC_TarEnc,Test_loss_TarEnc,Test_AUC_TarEnc = Decision_Tree_Classifier(X_train_target_encoding,y_train,X_test_target_encoding,y_test)
# Response Encoding
best_depth_ResEnc,best_samples_split_ResEnc,Train_loss_ResEnc,Train_AUC_ResEnc,Test_loss_ResEnc,Test_AUC_ResEnc = Decision_Tree_Classifier(X_train_response_encoding,y_train,X_test_response_encoding,y_test)
# One Hot Encoding
best_depth_ohe,best_samples_split_ohe,Train_loss_ohe,Train_AUC_ohe,Test_loss_ohe,Test_AUC_ohe = Decision_Tree_Classifier(X_train_ohe,y_train,X_test_ohe,y_test)

In [49]:
summary_table = PrettyTable(["Model","Encoding", "Hyperparameter1", "Hyperparameter2", "Train_log_loss", "Train_roc_auc_score", "Test_log_loss", "Test_roc_auc_score"]) #heading

summary_table.add_row(["Decision Tree","Ordinal Encoding",best_depth_OrEnc,best_samples_split_OrEnc,round(Train_loss_OrEnc,3),round(Train_AUC_OrEnc,3),round(Test_loss_OrEnc,3),round(Test_AUC_OrEnc,3)])
summary_table.add_row(["Decision Tree","Frequency Encoding",best_depth_FreEnc,best_samples_split_FreEnc,round(Train_loss_FreEnc,3),round(Train_AUC_FreEnc,3),round(Test_loss_FreEnc,3),round(Test_AUC_FreEnc,3)])
summary_table.add_row(["Decision Tree","Target Encoding",best_depth_TarEnc,best_samples_split_TarEnc,round(Train_loss_TarEnc,3),round(Train_AUC_TarEnc,3),round(Test_loss_TarEnc,3),round(Test_AUC_TarEnc,3)])
summary_table.add_row(["Decision Tree","Response Encoding",best_depth_ResEnc,best_samples_split_ResEnc,round(Train_loss_ResEnc,3),round(Train_AUC_ResEnc,3),round(Test_loss_ResEnc,3),round(Test_AUC_ResEnc,3)])
summary_table.add_row(["Decision Tree","One Hot Encoding",best_depth_ohe,best_samples_split_ohe,round(Train_loss_ohe,3),round(Train_AUC_ohe,3),round(Test_loss_ohe,3),round(Test_AUC_ohe,3)])

table = pd.read_html(summary_table.get_html_string())
Decision_Tree_Result = table[0]
Decision_Tree_Result

,Model,Encoding,Hyperparameter1,Hyperparameter2,Train_log_loss,Train_roc_auc_score,Test_log_loss,Test_roc_auc_score
0,Decision Tree,Ordinal Encoding,10,100,0.519,0.817,0.596,0.762
1,Decision Tree,Frequency Encoding,50,100,0.513,0.821,0.664,0.753
2,Decision Tree,Target Encoding,10,500,0.566,0.774,0.583,0.763
3,Decision Tree,Response Encoding,50,500,0.566,0.774,0.584,0.758
4,Decision Tree,One Hot Encoding,50,100,0.504,0.826,0.745,0.766


### Random Forest

In [50]:
def Random_Forest_Classifier(x_train,y_train,x_test,y_test):
  """This function returns best hyperparameter, train and test log_loss & roc_auc_score of Random_Forest Model"""

  clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None,max_features='log2',min_samples_leaf=3,random_state=42, n_jobs=-1)
  parameters = {'max_depth':[20, 50, 100], 'n_estimators':[1000,2000]}
  model = RandomizedSearchCV(clf, parameters, cv=5, scoring='roc_auc') #scoring='roc_auc' or 'neg_log_loss'
  model.fit(x_train, y_train)
  best_depth = model.best_params_['max_depth']
  best_n_estimators = model.best_params_['n_estimators']

  clf = RandomForestClassifier(n_estimators=best_n_estimators,criterion='gini',max_depth=best_depth,max_features='log2',min_samples_leaf=3, random_state=42, n_jobs=-1)
  clf.fit(x_train, y_train)

  Train_loss = log_loss(y_train,clf.predict_proba(x_train))
  Train_AUC = roc_auc_score(y_train,clf.predict_proba(x_train)[:,1])
  Test_loss = log_loss(y_test,clf.predict_proba(x_test))
  Test_AUC = roc_auc_score(y_test,clf.predict_proba(x_test)[:,1])

  return best_depth,best_n_estimators,Train_loss,Train_AUC,Test_loss,Test_AUC

In [ ]:
# Ordinal Encoding
best_depth_OrEnc,best_n_estimators_OrEnc,Train_loss_OrEnc,Train_AUC_OrEnc,Test_loss_OrEnc,Test_AUC_OrEnc = Random_Forest_Classifier(X_train_Ordinal_encoding,y_train,X_test_Ordinal_encoding,y_test)
# Frequency Encoding
best_depth_FreEnc,best_n_estimators_FreEnc,Train_loss_FreEnc,Train_AUC_FreEnc,Test_loss_FreEnc,Test_AUC_FreEnc = Random_Forest_Classifier(X_train_frequency_encoding,y_train,X_test_frequency_encoding,y_test)
# Target Encoding
best_depth_TarEnc,best_n_estimators_TarEnc,Train_loss_TarEnc,Train_AUC_TarEnc,Test_loss_TarEnc,Test_AUC_TarEnc = Random_Forest_Classifier(X_train_target_encoding,y_train,X_test_target_encoding,y_test)
# Response Encoding
best_depth_ResEnc,best_n_estimators_ResEnc,Train_loss_ResEnc,Train_AUC_ResEnc,Test_loss_ResEnc,Test_AUC_ResEnc = Random_Forest_Classifier(X_train_response_encoding,y_train,X_test_response_encoding,y_test)
# One Hot Encoding
best_depth_ohe,best_n_estimators_ohe,Train_loss_ohe,Train_AUC_ohe,Test_loss_ohe,Test_AUC_ohe = Random_Forest_Classifier(X_train_ohe,y_train,X_test_ohe,y_test)

In [ ]:
summary_table = PrettyTable(["Model","Encoding", "Hyperparameter1", "Hyperparameter2", "Train_log_loss", "Train_roc_auc_score", "Test_log_loss", "Test_roc_auc_score"]) #heading

summary_table.add_row(["Random Forest","Ordinal Encoding",best_depth_OrEnc,best_n_estimators_OrEnc,round(Train_loss_OrEnc,3),round(Train_AUC_OrEnc,3),round(Test_loss_OrEnc,3),round(Test_AUC_OrEnc,3)])
summary_table.add_row(["Random Forest","Frequency Encoding",best_depth_FreEnc,best_n_estimators_FreEnc,round(Train_loss_FreEnc,3),round(Train_AUC_FreEnc,3),round(Test_loss_FreEnc,3),round(Test_AUC_FreEnc,3)])
summary_table.add_row(["Random Forest","Target Encoding",best_depth_TarEnc,best_n_estimators_TarEnc,round(Train_loss_TarEnc,3),round(Train_AUC_TarEnc,3),round(Test_loss_TarEnc,3),round(Test_AUC_TarEnc,3)])
summary_table.add_row(["Random Forest","Response Encoding",best_depth_ResEnc,best_n_estimators_ResEnc,round(Train_loss_ResEnc,3),round(Train_AUC_ResEnc,3),round(Test_loss_ResEnc,3),round(Test_AUC_ResEnc,3)])
summary_table.add_row(["Random Forest","One Hot Encoding",best_depth_ohe,best_n_estimators_ohe,round(Train_loss_ohe,3),round(Train_AUC_ohe,3),round(Test_loss_ohe,3),round(Test_AUC_ohe,3)])

table = pd.read_html(summary_table.get_html_string())
Random_Forest_Classifier_Result = table[0]
Random_Forest_Classifier_Result